In [1]:
import numpy as np
import scipy

# Numerische Quadratur
Definition der Quadratur (Approximation eines Integrals), wobei c die Stützstellen sind und w die Gewichte.
$$
\int_{a}^{b} f(x) \, dx \approx Q_n(f; a, b) :=  \sum_{i=1}^{n} w_i f(c_i).
$$
Definition des Fehlers einer Quadratur
$$
E(n) := \left| \int_{a}^{b} f(x) \, dx - Q_n(f; a, b) \right|
$$
Definition der algebraischen Konvergenz
$E(n) = O\left( \frac{1}{n^p} \right)$

Definition der exponentiellen Konvergenz
$E(n) = O\left( q^n \right)$ mit $0 \leq q < 1$


## 5.4 Äquidistante Stützstellen

### Einfache Regeln zum Approximieren
### Theorem zur Konvergenz
$$
\left| \int_{I} f(t) \, dt - Q(f) \right| \leq C h^p \, \|f^{(p)} \|_{L^\infty(I)}
$$
Der Integrationsfehler hängt von der $p$-ten Ableitung der Funktion $f$ ab.
Die Schranke für den Fehler ist nur anwendbar, falls $f \in C^p(I)$ gilt, also $f$ $p$-fach stetig differenzierbar ist. Ist $f$ hinreichend differenzierbar ($f \in C^p$), so erzielt das Verfahren seine optimale Genauigkeit.
Fehlt jedoch diese Differenzierbarkeitsstufe ($f \notin C^p$), bleibt die erwartete Höchstleistung unerreicht und die Konvergenzgeschwindigkeit liegt unter $p$.

Mittelpunkt-Regel
$$
Q^{M}(f; a, b) = (b-a) f \left( \frac{a+b}{2} \right)
$$


In [1]:
def midpoint(f, a, b, N):
    """
    Midpoint quadrature rule for numerical integration.
    
    Parameters:
    f: Function to integrate, f(x)
    a: Lower bound of integration interval
    b: Upper bound of integration interval
    N: Number of subintervals
    
    Returns:
    I: Approximation of the integral using the midpoint rule
    """
    # Create N+1 equally spaced points from a to b and get step size h
    x, h = np.linspace(a, b, int(N) + 1, retstep=True)
    # Calculate midpoints between consecutive points and evaluate f at these midpoints
    # Sum all contributions: h * f(midpoint) for each subinterval
    I = np.sum(h * f((x[:-1] + x[1:]) / 2))
    return I


Trapez-Regel
$$
Q^{T}(f; a, b) = \frac{b-a}{2} (f(a) + f(b))
$$


In [ ]:
def trapezoidal(f, a, b, N):
    """
    Trapezoidal quadrature of function f from a to b with N subintervals.
    f: Function f(x)
    a, b: Bounds of the integration interval
    N: Number of subintervals
    """
    x, h = np.linspace(a, b, int(N)+1, retstep=True)
    # quadrature weights:
    # boundary nodes: w=1/2
    # internal nodes: w=1
    I = h/2.0 * (f(x[0]) + 2.0*sum(f(x[1:-1])) + f(x[-1]))
    return I

Simpson-Regel
$$
Q^{S}(f; a, b) = \frac{b-a}{6} \left ( f(a) + 4 f\left (\frac{a+b}{2} \right) + f(b) \right)
$$

In [ ]:
def simpson(f, a, b, N):
    """
    Simpson quadrature of function f from a to b with N subintervals.
    f: Function f(x)
    a, b: Bounds of the integration interval
    N: Number of subintervals
    this method uses 2*N+1 function evaluations
    """
    x, h = np.linspace(a, b, 2*int(N)+1, retstep=True)
    # quadrature weights:
    # boundary nodes: w=1/3
    # internal nodes: w=2/3
    # midpoint nodes: w=4/3
    I = h/3.0 * sum(f(x[:-2:2]) + 4.0*f(x[1:-1:2]) + f(x[2::2]))
    return I

In [ ]:
# !TODO Romberg Schema

## 5.5 Nicht äquidistante Stützstellen
### Definition Orthogonale Polynome:
Für eine Gewichtsfunktion $\omega(x)$ definieren wir das Skalarprodukt wie folgt.
$$
\langle f, g \rangle := \int_{a}^{b} f(x) g(x) \, \omega(x) \, dx
$$
Theorem: Für ein wie oben definiertes Skalarprodukt existiert eine eindeutige Folge
von Polynomen $p0$, $p1$, $p2$, . . . sodass diese paarweise orthogonal sind und $deg(pi) = i$
gilt.
#### Definition des Skalarprodukts für Legendre-Polynome
Die Gewichts funktion ist $omega(x) = 1$
$$
\langle f, g \rangle := \int_{-1}^{1} f(x) g(x) \, dx
$$

### Jacobi Polynome
Die Legendre-Polynome sind ein Spezialfall der Jacobi-Polynome.
Jacobi-Polynome sind orthogonal bezüglich des Skalarprodukts:
$$
\langle f, g \rangle := \int_{-1}^{1} f(x) g(x) (1-x)^\alpha (1+x)^\beta \, dx
$$
Durch geeignete Wahl von $\alpha$ und $\beta$ bekommt man neue Quadraturen.
####
### 5.5.1 Gaus Quadratur
**Motivation**:
Die Gauss-Quadratur macht keine Annahme über Abstände (also nicht äquidistant),
sondern sucht Punkte und Gewichte so, dass Polynome bis zum einem möglichst hohen
Grad exakt integriert werden

**Definition**
$$
\int_{-1}^{1} f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)
$$

#### Gaus-Legendre Quadratur
Die Gewichts funktion ist $omega(x) = 1$.

$\alpha = 0, \beta = 0$

**Theorem**: Die Knoten $x_1, . . . , x_n$, die diese Eigenschaft erfüllen, sind die Nullstellen
des Legendre-Polynoms $p_n(x)$.

In [ ]:
def gauss_legendre(f, a, b, n):
    r"""Integrate f, over [a, b], using Gauss-Legendre quadrature.

    f       : function to integrate
    a       : left end
    b       : right end
    n       : degree of the quadrature rule
    """
    xn, wn = scipy.special.roots_legendre(n)
    x = 0.5 * (b-a) * xn + (a+b) / 2
    w = 0.5 * (b-a) * wn
    return np.sum(w*f(x))


#### Gaus-Radau Quadratur
Die Gewichtsfunktion ist $omega(x) = 1$.
Falls der rechte Rand fixiert ist, gilt: $\alpha = 1, \beta = 0$, ansonsten falls der linke Rand fixiert ist: $\alpha = 0, \beta = 1$.

In [1]:
def gauss_radau(f, a, b, n, fixed='r'):
    """
    Integrate f over [a, b] using Gauss–Radau quadrature.

    Parameters
    ----------
    f : callable
        Function to integrate.
    a, b : float
        Integration interval.
    n : int
        Number of nodes (including the fixed one).
    fixed : {'r', 'l'}, optional
        Fixed endpoint:
        - 'r': right endpoint (x = 1) included.
        - 'l': left endpoint (x = -1) included.
    """
    if fixed == 'r':
        # Right endpoint fixed at x=1
        r, w = scipy.special.roots_jacobi(n-1, alpha=1, beta=0)
        nodes = np.hstack((r, [1.0]))
        weights = np.hstack((w / (1 - r), [2 / n**2]))
    else:
        # Left endpoint fixed at x=-1
        r, w = scipy.special.roots_jacobi(n-1, alpha=0, beta=1)
        nodes = np.hstack(([-1.0], r))
        weights = np.hstack(([2 / n**2], w / (1 + r)))

    # Map from [-1, 1] → [a, b]
    x = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    w = 0.5 * (b - a) * weights

    return np.sum(w * f(x))

#### Gaus-Lobatto Quadratur
Die Gewichtsfunktion ist $omega(x) = 1$.
Beide Ränder der Knoten sind fixiert. $\alpha = 1, \beta = 1$

In [2]:
def gauss_lobatto(f, a, b, n):
    """
    Integrate f over [a, b] using Gauss–Lobatto quadrature
    with n nodes (including endpoints), using roots_jacobi.

    Parameters
    ----------
    f : callable
        Function to integrate.
    a, b : float
        Integration interval.
    n : int
        Number of nodes (including the fixed one).
    """
    if n < 2:
        raise ValueError("Gauss–Lobatto requires at least 2 nodes")

    if n == 2:
        # Only endpoints
        nodes = np.array([-1.0, 1.0])
        weights = np.array([1.0, 1.0])
    else:
        # Internal nodes and weights from Jacobi(n-2,1,1)
        internal_nodes, internal_weights = scipy.special.roots_jacobi(n-2, alpha=1, beta=1)

        # All nodes
        nodes = np.hstack(([-1.0], internal_nodes, [1.0]))

        # All weights
        weights = np.zeros(n)
        weights[0] = 2 / (n * (n-1))      # left endpoint
        weights[-1] = 2 / (n * (n-1))     # right endpoint

        # Internal weights: scale Jacobi weights
        weights[1:-1] = internal_weights * 2 / (n*(n-1))

    # Map from [-1,1] → [a,b]
    x = 0.5*(b - a)*nodes + 0.5*(a + b)
    w = 0.5*(b - a)*weights

    return np.sum(w * f(x))